In [1]:
# one hot encoding

In [2]:
# housing iteration 3

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

housing_maindf = pd.read_csv('~/Documents/datacamp/WBS-School/wbs-housing-iter-0-2/iter-3/housing-classification-iter3.csv')
housing5 = pd.read_csv('~/Documents/datacamp/WBS-School/wbs-housing-iter-0-2/housing-classification-iter-0-2.csv')
housing6 = pd.read_csv('~/Documents/datacamp/WBS-School/wbs-housing-iter-0-2/iter-6/housing-classification-iter6.csv')
kaggle = pd.read_csv('~/Documents/datacamp/WBS-School/wbs-housing-iter-0-2/test-housing-classification.csv')


housing_df = housing6.copy()


In [4]:
# housing_df.info()

In [5]:
X = housing_df.copy()


In [6]:
# X = kaggle

In [7]:
# target
y = X.pop("Expensive")


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31416)


In [9]:
# imputing train set

In [10]:
# helper
def to_df(arr,df):
    print(arr.shape)
    print(df.shape)
    return pd.DataFrame(arr, columns=df.columns, index=df.index)



In [11]:
from sklearn.impute import SimpleImputer

# # selecting non-numerical columns
# X_train_cat = X_train.select_dtypes(exclude="number")

# # defining the imputer to use "N_A" as replacement value
# cat_imputer = SimpleImputer(strategy="constant", 
#                             fill_value="N_A")#.set_output(transform='pandas')

# # fitting and transforming
# X_cat_imputed = cat_imputer.fit_transform(X_train_cat)
# X_cat_imputed = to_df(X_cat_imputed,X_train_cat)


In [12]:
# # Selecting numerical columns
# X_train_num = X_train.select_dtypes(include="number")

# # Imputing the mean
# num_imputer = SimpleImputer(strategy="mean")#.set_output(transform='pandas')

# # Fitting and transforming
# X_num_imputed = num_imputer.fit_transform(X_train_num)
# X_num_imputed = to_df(X_num_imputed,X_train_num)
# X_num_imputed.head()

In [13]:
# # Concatenating all columns
# X_imputed = pd.concat([X_cat_imputed, X_num_imputed], axis=1)

# X_imputed.head()

In [14]:
# apply one hot encoder

In [15]:
from sklearn.preprocessing import OneHotEncoder

# # initialize
# # my_onehot = OneHotEncoder(drop="first",sparse_output=False).set_output(transform='pandas')
# my_onehot = OneHotEncoder()
# # fit
# my_onehot.fit(X_cat_imputed)

# # transform
# X_cat_imputed_mat = my_onehot.transform(X_cat_imputed)

In [16]:
# import scipy.sparse
# X_cat_imputed_onehot = pd.DataFrame.sparse.from_spmatrix(X_cat_imputed_mat)

In [17]:
# X_imputed = pd.concat([X_cat_imputed_onehot, X_num_imputed], axis=1)

# X_imputed.head(3)

In [18]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
stscaler = StandardScaler()
mmscaler = MinMaxScaler()

In [19]:
# preprocessor

In [20]:
from sklearn.pipeline import make_pipeline

# select categorical and numerical column names
X_cat_columns = X.select_dtypes(exclude="number").copy().columns
X_num_columns = X.select_dtypes(include="number").copy().columns

# create numerical pipeline, only with the SimpleImputer(strategy="mean")
numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean"),
    mmscaler
)
 
 # create categorical pipeline, with the SimpleImputer(fill_value="N_A") and the OneHotEncoder
categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    # OneHotEncoder(sparse_output=False)
    OneHotEncoder(handle_unknown='ignore')
)

In [21]:
# initialize transformer

In [22]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num_columns),
        ("cat_pipe", categoric_pipe, X_cat_columns),
    ]
)

In [23]:
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier


In [24]:
from sklearn.model_selection import GridSearchCV

dtclass = DecisionTreeClassifier()
dtreg = DecisionTreeRegressor()
full_pipeline = make_pipeline(preprocessor, 
                              # dtclass
                              dtreg
                             )

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    # "decisiontreeclassifier__max_depth": range(2, 14, 2),
    # "decisiontreeclassifier__min_samples_leaf": range(3, 12, 2),
    "decisiontreeregressor__max_depth": range(2, 14, 2),
    "decisiontreeregressor__min_samples_leaf": range(3, 12, 2),

}

search = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=10,
                      verbose=0,
                      scoring='accuracy'
                     )



In [ ]:
search.fit(X_train, y_train)

In [26]:
search.best_score_

nan

In [27]:
search.best_params_

{'columntransformer__num_pipe__simpleimputer__strategy': 'mean',
 'decisiontreeregressor__max_depth': 2,
 'decisiontreeregressor__min_samples_leaf': 3}

In [28]:
search.cv_results_['mean_test_score']

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan])

In [29]:
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  Index(['LotArea', 'LotFrontage', 'TotalBsmtSF', 'BedroomAbvGr', 'Fireplaces',
       'PoolArea', 'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'MSSubClass',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRem...
       'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtFinType2',
       'HeatingQC', 'Electrical', 'Functional', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fen

In [30]:
from sklearn.metrics import accuracy_score, roc_auc_score
y_train_pred = full_pipeline.predict(X_train)

accuracy_score(y_train, y_train_pred)

1.0

In [31]:
full_pipeline.fit(X_test, y_test)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  Index(['LotArea', 'LotFrontage', 'TotalBsmtSF', 'BedroomAbvGr', 'Fireplaces',
       'PoolArea', 'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'MSSubClass',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRem...
       'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtFinType2',
       'HeatingQC', 'Electrical', 'Functional', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fen

In [32]:
y_test_pred = full_pipeline.predict(X_test)

accuracy_score(y_test, y_test_pred)

1.0

In [33]:
# preds = full_pipeline.predict_proba(X_test)

# roc_auc_score(y_test, preds, multi_class="ovr")

In [34]:
from sklearn.metrics import mean_squared_error as MSE
# Compute mse_dt train set
mse_dtree = MSE(y_train, y_train_pred)

# Compute rmse_dt
rmse_dtree = mse_dtree**(1/2)

# Print rmse_dt
print("Train set RMSE of dtree: {:.2f}".format(rmse_dtree))

Train set RMSE of dtree: 0.00


In [35]:
# # Compute mse_dt test set
mse_dtree = MSE(y_test, y_test_pred)

# # Compute rmse_dt
rmse_dtree = mse_dtree**(1/2)

# # Print rmse_dt
print("Test set RMSE of dtree: {:.2f}".format(rmse_dtree))

Test set RMSE of dtree: 0.00


In [36]:
# final model params

In [37]:
# 'columntransformer__num_pipe__simpleimputer__strategy': 'median',
#  'decisiontreeclassifier__max_depth': 4,
#  'decisiontreeclassifier__min_samples_leaf': 5

In [38]:
fin_preprocessor = preprocessor

findtree = DecisionTreeClassifier(
    max_depth= 4,
    min_samples_leaf=5
)

kaggle_pipeline = make_pipeline(fin_preprocessor, findtree)



In [39]:
print(len(list(kaggle.columns)))


80


In [40]:
feature_columns = list(X_train.columns)
# feature_columns.remove('target')
# feature_columns.remove('Id')
print(len(feature_columns))
print('all features for training ->',feature_columns)

80
all features for training -> ['LotArea', 'LotFrontage', 'TotalBsmtSF', 'BedroomAbvGr', 'Fireplaces', 'PoolArea', 'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'MSZoning', 'Condition1', 'Heating', 'Street', 'CentralAir', 'Foundation', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'KitchenQual', 'FireplaceQu', 'MSSubClass', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'KitchenAbvGr', 'TotRmsAbvGrd', 'GarageYrBlt', 'GarageArea', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'MiscVal', 'MoSold', 'YrSold', 'Id', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtFinType2', 'HeatingQC', 'Electrical', 'Functional', 'GarageType', 'Ga

In [41]:
# remove columns from kaggle dataset
drop_ls = [x for x in kaggle.columns if x not in feature_columns]

kaggle.drop(drop_ls, axis=1, inplace=True)
print(len(list(kaggle.columns)))


80


In [42]:
kaggle.shape


(1459, 80)

In [43]:
kaggle_pipeline.fit(X_train,y_train)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  Index(['LotArea', 'LotFrontage', 'TotalBsmtSF', 'BedroomAbvGr', 'Fireplaces',
       'PoolArea', 'GarageCars', 'WoodDeckSF', 'ScreenPorch', 'MSSubClass',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRem...
       'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtFinType2',
       'HeatingQC', 'Electrical', 'Functional', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fen

In [44]:
to_predict = kaggle
fin_pred = kaggle_pipeline.predict(to_predict)

# accuracy_score(y_train, kaggle_pred)

In [45]:
from datetime import datetime

now = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
my_submission = pd.DataFrame({'Id': to_predict.index, 'Expensive': fin_pred})

my_submission.to_csv(f'talladen_submit_{now}.csv', index=False)